# Extracting data from Flopy files 

notes from 
- https://codesachin.wordpress.com/2016/06/09/the-magic-behind-attribute-access-in-python/
- https://stackoverflow.com/questions/5969806/print-all-properties-of-a-python-class

# Data from 
### Preliminary Tutuila Groundwater Optimization 
### Chris Shuler CEE 696 Final Project

this cell loads modules and loads up the flopy data

In [ ]:
%matplotlib notebook
import scipy.optimize as opt
import numpy as np
import flopy
import flopy.utils.binaryfile as bf
import matplotlib.pyplot as plt
import inspect


# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))


# Assign name and create modflow model object
txtname = 'test_no_5'
model = flopy.modflow.Modflow(txtname, exe_name='mf2005')

dis = flopy.modflow.ModflowDis.load('TutCoast_simple5.dis', model)       # this command is neededed to load an existing .dis file   (essentially is grid geometry)
bas = flopy.modflow.ModflowBas.load('TutCoast_simple5.ba6', model)        # add an existing .basic package                         (essentially is ibound and starting heads)
lpf = flopy.modflow.ModflowLpf.load('TutCoast_simple5.lpf', model)       # add an existing .lpf file     (hydraulic conductivity)
rch = flopy.modflow.ModflowRch.load('TutCoast_simple5.rch', model)       # addd the Recharge package
wel = flopy.modflow.ModflowWel.load('TutCoast_simple5.wel', model)       # addd the well package
hobs = flopy.modflow.ModflowHob.load('TutCoast_simple5.hob', model)      # addd the obervations well package
pcg = flopy.modflow.ModflowPcg(model)  

# Add OC package to the MODFLOW model
spd = {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
oc = flopy.modflow.ModflowOc(model, stress_period_data=spd, compact=True)# gradient solver package from flopy


# Now lets explore the data within the packages themselves

In [ ]:
#this tries to list any attribute that is possible to reach.
dir(bas) 

In [ ]:
# using .__dict__ on a class will create a dictionary of  all the attributes defined for the object itself. It maps the attribute name to its value.
bas.__dict__ 
# or seems to do the same thing
vars(bas)

In [ ]:
#to acess values in the package
bas.allowDuplicates

In [ ]:
#to acess values in the package
lpf.ipakcb

In [ ]:
#to access arrays in the package
bas.strt.array

In [ ]:
# see the shape
np.shape(bas.strt.array)

In [ ]:
# first value is the layer number here there is only one, access it with []
np.shape(bas.strt.array[0])

In [ ]:
#plot the data for the given layer
arr = bas.strt.array

layer = 0

data = arr[layer]

fig, ax = plt.subplots(figsize = (5,3))
im = plt.imshow(data,  aspect=0.5)
plt.colorbar().set_label('label', rotation=270)
plt.tight_layout()

# Lets try and run the original model

In [ ]:
# Write the MODFLOW model input files
model.write_input()

# Run the MODFLOW model
success, buff = model.run_model()

# display the head distribution 

In [ ]:
# Post process the results

hds = bf.HeadFile(txtname + '.hds')
head = hds.get_data(totim=1.0)
levels = np.linspace(0, 10, 11)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1, 1, 1, aspect='equal')

hds = bf.HeadFile(txtname+'.hds')
times = hds.get_times()
head = hds.get_data(totim=times[-1])
levels = np.linspace(0, 10, 11)

cbb = bf.CellBudgetFile(txtname+'.cbc')
kstpkper_list = cbb.get_kstpkper()
frf = cbb.get_data(text='FLOW RIGHT FACE', totim=times[-1])[0]
fff = cbb.get_data(text='FLOW FRONT FACE', totim=times[-1])[0]

modelmap = flopy.plot.ModelMap(model=model, layer=0)
qm = modelmap.plot_ibound()
lc = modelmap.plot_grid()
cs = modelmap.contour_array(head, levels=levels)
quiver = modelmap.plot_discharge(frf, fff, head=head)

## LPF class 

In [ ]:
# Define which modules to work on
The_flopy_class = flopy.modflow.ModflowLpf # this is the flopy class that you want to operate on
Local_Object_for_class = lpf


### Step 1  - access the default values for each module, and assign all of them as local variables in this notebook

defualt_dict = {} 
goodlist = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).args[2:]                    # this is the list of variables omitting the self and model ones since they dont have defaults
for idx, key in enumerate(goodlist):
    defualt_dict[key] = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).defaults[idx]  # assign each variable name as a key, each default as a value  in a dictionary 
    
# MAGIC!  this converts a dictionaries keys to local variables that are attached to the dictionary datas 
for key,val in defualt_dict.items():
        exec(key + '=val')

### Step 2  - extract the values from the GMS file and write them to local variables in memory, those that match those above are overwrittten

# MAGIC!  this converts a dictionaries keys to local variables that are attached to the dictionary datas 
module_dictionary = Local_Object_for_class.__dict__ 
for key,val in module_dictionary.items():
    exec(key + '=val')

list_of_vars = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).args   # this will make a list of all the valid variables for the module 

# print out a string to paste into the flopy module definition  !!! remember to ditch the last  comma when pasting!!!!
copypastestring = ""  
for i in list_of_vars:
    variable = i+" = "+i+", "
    copypastestring = copypastestring + variable
copypastestring = copypastestring.replace('self = self, ' , '')   # pull out the self variable 
copypastestring

In [ ]:
lpf = flopy.modflow.ModflowLpf(model = model, laytyp = laytyp, layavg = layavg, chani = chani, layvka = layvka, laywet = laywet, 
                               ipakcb = ipakcb, hdry = hdry, iwdflg = iwdflg, wetfct = wetfct, iwetit = iwetit, ihdwet = ihdwet,
                               hk = 1, hani = hani, vka = vka, ss = ss, sy = sy, vkcb = vkcb, wetdry = wetdry, 
                               storagecoefficient = storagecoefficient, constantcv = constantcv, thickstrt = thickstrt, 
                               nocvcorrection = nocvcorrection, novfc = novfc, extension = extension, unitnumber = unitnumber, 
                               filenames = filenames)

        

In [ ]:

# Write the MODFLOW model input files
model.write_input()

# Run the MODFLOW model
success, buff = model.run_model()

# Post process the results

hds = bf.HeadFile(txtname + '.hds')
head = hds.get_data(totim=1.0)
levels = np.linspace(0, 10, 11)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1, 1, 1, aspect='equal')

hds = bf.HeadFile(txtname+'.hds')
times = hds.get_times()
head = hds.get_data(totim=times[-1])
levels = np.linspace(0, 10, 11)

cbb = bf.CellBudgetFile(txtname+'.cbc')
kstpkper_list = cbb.get_kstpkper()
frf = cbb.get_data(text='FLOW RIGHT FACE', totim=times[-1])[0]
fff = cbb.get_data(text='FLOW FRONT FACE', totim=times[-1])[0]

modelmap = flopy.plot.ModelMap(model=model, layer=0)
qm = modelmap.plot_ibound()
lc = modelmap.plot_grid()
cs = modelmap.contour_array(head, levels=levels)
quiver = modelmap.plot_discharge(frf, fff, head=head)

print(np.max(head))

# Strange behavior, if trying to change more than one package at once the run fails not sure why, but shouldnt need to change more than one anyway...


In [ ]:
## DIS class

In [ ]:
# Define which modules to work on

The_flopy_class = flopy.modflow.ModflowDis # this is the flopy class that you want to operate on
Local_Object_for_class = dis


### Step 1  - access the default values for each module, and assign all of them as local variables in this notebook

defualt_dict = {} 
goodlist = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).args[2:]                    # this is the list of variables omitting the self and model ones since they dont have defaults
for idx, key in enumerate(goodlist):
    defualt_dict[key] = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).defaults[idx]  # assign each variable name as a key, each default as a value  in a dictionary 
    
# MAGIC!  this converts a dictionaries keys to local variables that are attached to the dictionary datas 
for key,val in defualt_dict.items():
        exec(key + '=val')

### Step 2  - extract the values from the GMS file and write them to local variables in memory, those that match those above are overwrittten

# MAGIC!  this converts a dictionaries keys to local variables that are attached to the dictionary datas 
module_dictionary = Local_Object_for_class.__dict__ 
for key,val in module_dictionary.items():
    exec(key + '=val')

list_of_vars = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).args   # this will make a list of all the valid variables for the module 

# print out a string to paste into the flopy module definition  !!! remember to ditch the last  comma when pasting!!!!
copypastestring = ""  
for i in list_of_vars:
    variable = i+" = "+i+", "
    copypastestring = copypastestring + variable
copypastestring = copypastestring.replace('self = self, ' , '')   # pull out the self variable 
copypastestring

In [ ]:
dis = flopy.modflow.ModflowDis(model = model, nlay = nlay, nrow = nrow, ncol = ncol, nper = nper, 
                               delr = delr, delc = delc, laycbd = laycbd, top = top, botm = botm, 
                               perlen = perlen, nstp = nstp, tsmult = tsmult, steady = steady, itmuni = itmuni, 
                               lenuni = lenuni, extension = extension, unitnumber = unitnumber, filenames = filenames, 
                               xul = xul, yul = yul, rotation = rotation, proj4_str = proj4_str, start_datetime = start_datetime)

## Bas Class



In [ ]:
# Define which modules to work on

The_flopy_class = flopy.modflow.ModflowBas # this is the flopy class that you want to operate on
Local_Object_for_class = bas


### Step 1  - access the default values for each module, and assign all of them as local variables in this notebook

defualt_dict = {} 
goodlist = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).args[2:]                    # this is the list of variables omitting the self and model ones since they dont have defaults
for idx, key in enumerate(goodlist):
    defualt_dict[key] = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).defaults[idx]  # assign each variable name as a key, each default as a value  in a dictionary 
    
# MAGIC!  this converts a dictionaries keys to local variables that are attached to the dictionary datas 
for key,val in defualt_dict.items():
        exec(key + '=val')

### Step 2  - extract the values from the GMS file and write them to local variables in memory, those that match those above are overwrittten

# MAGIC!  this converts a dictionaries keys to local variables that are attached to the dictionary datas 
module_dictionary = Local_Object_for_class.__dict__ 
for key,val in module_dictionary.items():
    exec(key + '=val')

list_of_vars = inspect.getfullargspec(The_flopy_class.__dict__["__init__"]).args   # this will make a list of all the valid variables for the module 

# print out a string to paste into the flopy module definition  !!! remember to ditch the last  comma when pasting!!!!
copypastestring = ""  
for i in list_of_vars:
    variable = i+" = "+i+", "
    copypastestring = copypastestring + variable
copypastestring = copypastestring.replace('self = self, ' , '')   # pull out the self variable 
copypastestring

In [ ]:
bas = flopy.modflow.ModflowBas(model = model, ibound = ibound, strt = strt, ifrefm = ifrefm,
                               ixsec = ixsec, ichflg = ichflg, stoper = stoper, hnoflo = hnoflo, extension = extension, 
                               unitnumber = unitnumber, filenames = filenames)

In [ ]:
# Assign name and create modflow model object
txtname = 'test_no_5'
model = flopy.modflow.Modflow(txtname, exe_name='mf2005')

dis = flopy.modflow.ModflowDis.load('TutCoast_simple5.dis', model)       # this command is neededed to load an existing .dis file   (essentially is grid geometry)
bas = flopy.modflow.ModflowBas.load('TutCoast_simple5.ba6', model)        # add an existing .basic package                         (essentially is ibound and starting heads)
lpf = flopy.modflow.ModflowLpf.load('TutCoast_simple5.lpf', model)       # add an existing .lpf file     (hydraulic conductivity)
rch = flopy.modflow.ModflowRch.load('TutCoast_simple5.rch', model)       # addd the Recharge package
wel = flopy.modflow.ModflowWel.load('TutCoast_simple5.wel', model)       # addd the well package
hobs = flopy.modflow.ModflowHob.load('TutCoast_simple5.hob', model)      # addd the obervations well package
pcg = flopy.modflow.ModflowPcg(model)  

# Add OC package to the MODFLOW model
spd = {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
oc = flopy.modflow.ModflowOc(model, stress_period_data=spd, compact=True)# gradient solver package from flopy
